<a href="https://colab.research.google.com/github/vsaben/demystifying-mlops-pluralsight/blob/main/prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. DATA

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
automobile_df = pd.read_csv("auto-mpg.csv")

automobile_df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino


In [4]:
automobile_df.shape

(398, 9)

In [6]:
automobile_df.dtypes

mpg             float64
cylinders         int64
displacement    float64
horsepower       object
weight            int64
acceleration    float64
model year        int64
origin            int64
car name         object
dtype: object

In [7]:
automobile_df["horsepower"] = pd.to_numeric(automobile_df["horsepower"], errors="coerce")

In [8]:
automobile_df.isnull().sum()

mpg             0
cylinders       0
displacement    0
horsepower      6
weight          0
acceleration    0
model year      0
origin          0
car name        0
dtype: int64

In [9]:
automobile_df = automobile_df.fillna(automobile_df.mean())

<ipython-input-9-2b3e8e8759ec>:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  automobile_df = automobile_df.fillna(automobile_df.mean())


In [10]:
automobile_df.isna().sum()

mpg             0
cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
model year      0
origin          0
car name        0
dtype: int64

In [11]:
automobile_df = automobile_df.drop(['car name'], axis = 1)

In [12]:
automobile_df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin
0,18.0,8,307.0,130.0,3504,12.0,70,1
1,15.0,8,350.0,165.0,3693,11.5,70,1
2,18.0,8,318.0,150.0,3436,11.0,70,1
3,16.0,8,304.0,150.0,3433,12.0,70,1
4,17.0,8,302.0,140.0,3449,10.5,70,1


In [29]:
X = automobile_df.drop(['mpg'], axis = 1)
Y = automobile_df['mpg']

# 2. MODEL

In [30]:
# backlog: figure out how to normaliza data in sklearn.

from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics

In [32]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(X, Y, test_size = 0.2)

# x_train = preprocessing.normalize(x_train)
lm = linear_model.LinearRegression().fit(x_train, y_train)
y_pred = lm.predict(x_test)

print("R2: ", lm.score(x_train, y_train))
print("MSE: ", metrics.mean_squared_error(y_test, y_pred))

R2:  0.8216895601607395
MSE:  10.080292178600791


In [34]:
x_sample = [6, 117, 100, 3000, 15, 85, 1]
print(lm.predict([x_sample]))

[27.15698002]


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


# SERIALISATION

In [35]:
import pickle

In [36]:
pickle.dump(lm, open('model.pkl', 'wb'))

In [37]:
model = pickle.load(open('model.pkl', 'rb'))

# DEPLOYMENT

In [43]:
!pip install flask-ngrok
!pip install pyngrok==4.1.1

  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15963 sha256=3edc823a59b707c7e45b61e89ddb72a9d02032743fd1d6ebbb6b643768bb8a36
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902c91a8
Successfully built pyngrok


In [44]:
import flask
import flask_ngrok
import requests
import numpy as np

app = flask.Flask(__name__)
flask_ngrok.run_with_ngrok(app)

@app.route('/predict', methods=['POST'])
def home():
  cylinders = int(flask.request.args.get('cylinders', ''))
  displacement = int(flask.request.args.get('displacement', ''))
  horsepower = int(flask.request.args.get('horsepower', ''))
  weight = int(flask.request.args.get('weight', ''))
  acceleration = int(flask.request.args.get('acceleration', ''))
  modelYear = int(flask.request.args.get('modelYear', ''))
  origin = int(flask.request.args.get('origin', ''))
  prediction = model.predict([[cylinders, displacement, horsepower, weight, acceleration, modelYear, origin]])
  print(prediction)

  return 'Prediction is ' + str(prediction[0])

app.run()

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://1fdd-34-30-237-172.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
INFO:werkzeug:127.0.0.1 - - [27/Oct/2023 15:07:46] "POST /predict?cylinders=6&displacement=117&horsepower=100&weight=3000&acceleration=15&modelYear=85&origin=1 HTTP/1.1" 200 -


[27.15698002]
